In [ ]:
import pandas as pd
from tqdm import tqdm

In [ ]:
from zeep import Client
import hashlib

wsdl = "https://www.brenda-enzymes.org/soap/brenda_zeep.wsdl"
password = hashlib.sha256("bioretro".encode("utf-8")).hexdigest()
client = Client(wsdl)
parameters = ( "itail@mit.edu",password, "ecNumber*1.1", "sequence*", "noOfAminoAcids*", "firstAccessionCode*", "source*", "id*", "organism*")
              
#               "ecNumber*1.1.1.1","organism*Homo sapiens","kmValue*",
#               "kmValueMaximum*","substrate*","commentary*","ligandStructureId*","literature*" )
#               "22834")
# resultString = client.service.getReferenceById(*parameters)
resultString = client.service.getSequence(*parameters)#.getKmValue(*parameters)
print (resultString)

In [ ]:
import pandas as pd

In [ ]:
bkms_df = pd.read_csv('bkms/1Sep2023_bkms-mapped.txt', sep='\t')

In [ ]:
brenda_subset_df = bkms_df[~bkms_df['Reaction_ID_BRENDA'].isna()]

In [ ]:
# check that almost all entries with no e.c. number are duplicates
brenda_w_ec = brenda_subset_df[~brenda_subset_df['EC_Number'].isna()]
brenda_wo_ec = brenda_subset_df[brenda_subset_df['EC_Number'].isna()]
len(brenda_wo_ec[brenda_wo_ec['Reaction'].map(lambda x : x in brenda_w_ec['Reaction'].values)])
# brenda_subset_df[brenda_subset_df['EC_Number'].isna()]

In [ ]:
brenda_w_ec

In [ ]:
wsdl = "https://www.brenda-enzymes.org/soap/brenda_zeep.wsdl"
password = hashlib.sha256("bioretro".encode("utf-8")).hexdigest()
client = Client(wsdl)
parameters = ( "itail@mit.edu",password, "ecNumber*5.3.3.3", "sequence*", "noOfAminoAcids*", "firstAccessionCode*", "source*", "id*", "organism*")
              
#               "ecNumber*1.1.1.1","organism*Homo sapiens","kmValue*",
#               "kmValueMaximum*","substrate*","commentary*","ligandStructureId*","literature*" )
#               "22834")
# resultString = client.service.getReferenceById(*parameters)
resultString = client.service.getSequence(*parameters)#.getKmValue(*parameters)
print (resultString)

In [ ]:
wsdl = "https://www.brenda-enzymes.org/soap/brenda_zeep.wsdl"
password = hashlib.sha256("bioretro".encode("utf-8")).hexdigest()
client = Client(wsdl)
parameters = ( "itail@mit.edu",password, "ecNumber*5.3.3.3","organism*", "reaction*", "commentary*", "literature*")
              
#               "ecNumber*1.1.1.1","organism*Homo sapiens","kmValue*",
#               "kmValueMaximum*","substrate*","commentary*","ligandStructureId*","literature*" )
#               "22834")
# resultString = client.service.getReferenceById(*parameters)
resultString = client.service.getReaction(*parameters)#.getKmValue(*parameters)
print (resultString)

In [ ]:

wsdl = "https://www.brenda-enzymes.org/soap/brenda_zeep.wsdl"
password = hashlib.sha256("bioretro".encode("utf-8")).hexdigest()
client = Client(wsdl)
parameters = ( "itail@mit.edu",password, "ecNumber*5.3.3.3","role*", "ligand*", "organism*", "ligandStructureId*")
              
#               "ecNumber*1.1.1.1","organism*Homo sapiens","kmValue*",
#               "kmValueMaximum*","substrate*","commentary*","ligandStructureId*","literature*" )
#               "22834")
# resultString = client.service.getReferenceById(*parameters)
resultString = client.service.getLigands(*parameters)#.getKmValue(*parameters)
print (resultString)

In [ ]:
parameters = ("itail@mit.edu",password,"ecNumber*5.3.3.3", "substrate*", "reactionPartners*", "organism*", "ligandStructureId*")
resultString = client.service.getSubstrate(*parameters)
print (resultString)

In [ ]:
parameters = ("itail@mit.edu",password,"ecNumber*5.3.3.3", "substrates*", 
              "commentarySubstrates*", "literatureSubstrates*", "organismSubstrates*", "products*", 
              "commentaryProducts*", "literatureProducts*", "organismProducts*", "reversibility*", "reversibility*")
resultString = client.service.getSubstratesProducts(*parameters)
print (resultString)

In [ ]:
parameters = ("itail@mit.edu",password,"ecNumber*5.3.3.3", "naturalSubstrates*",
              "organismNaturalSubstrates*", "commentaryNaturalSubstrates*",
              "naturalProducts*", "commentaryNaturalProducts*", "organismNaturalProducts*",
              "reversibility*", "reversibility*")
resultString = client.service.getNaturalSubstratesProducts(*parameters)
print (resultString)

In [ ]:
import json

In [ ]:
with open('/Users/Itai/Downloads/brenda_2023_1 3.json','r') as f:
    brenda_flatfile = json.load(f)

In [ ]:
brenda_flatfile['data'].keys()

In [ ]:
brenda_flatfile['data']['4.2.3.21'].keys()

In [ ]:
brenda_flatfile['data']['4.2.3.21']['reaction']

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
url = 'https://www.brenda-enzymes.org/enzyme.php?ecno=4.2.3.21#SUBSTRATE'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')
[s.get('span') for s in soup.find_all('div')]
# soup

In [ ]:
type(soup.find_all('div'))

In [ ]:
[x for x in soup.find_all('div', class_='cell') if 'trans,trans' in x.text]

In [ ]:
for tag in soup.find_all('div'):
    if 'trans,trans-farnesyl diphosphate' in tag.text and 'tab' in tag.text and not '1.1.1.1' in tag.text:
        if tag.get('id') == 'tab37':
            print ('~~~~~~~~~~~~~~~~~')
            print ([x.text+'\n' for x in tag.find_all('div')])

In [ ]:
# Find all the div elements with class 'hidden'
hidden_divs = soup.find_all('div')#, class_='hidden')

for div in hidden_divs:
    # Extract values from the first, second, and fifth columns
    cell0 = div.find('div', class_='cell', id=lambda x: x and x.endswith('c0'))
    cell1 = div.find('div', class_='cell', id=lambda x: x and x.endswith('c1'))
    cell4 = div.find('div', class_='cell', id=lambda x: x and x.endswith('c4'))

    if cell0 and cell1 and cell4:
        value0 = cell0.text.strip()
        value1 = cell1.text.strip()
        value4 = cell4.text.strip()

        print(f"Column 0: {value0}\nColumn 1: {value1}\nColumn 4: {value4}\n")

In [ ]:
hidden_divs

In [ ]:
soup.find('table')

In [ ]:
requests.get('https://www.brenda-enzymes.org/sequences.php?AC=F8WQD0').text

In [ ]:
import re
class BrendaPage():
   
    def __init__(self, url):
        res = requests.get(url)
        self.soup = BeautifulSoup(res.text, 'html.parser')
        self.get_table_headers()
        self.get_substrate_table_info()

    def get_table_headers(self):
        # return a dictionary of table numbers with a least of the headers for that table
        header_divs = self.soup.find_all('div',id=re.compile('tab[0-9]+_head'))
        self.table_headers_dict = {}
        for tab_div in header_divs:
            tab_num = int(re.findall('(?<=tab)[0-9]+(?=_head)', tab_div.get('id'))[0])
            self.table_headers_dict[tab_num] = [x.strip() for x in tab_div.text.split('\n') if len(x.strip())]

    def _get_substrates_table_id(self):
        #return id for SUBSTRATES table
        titles = ['SUBSTRATE','PRODUCT','REACTION DIAGRAM','ORGANISM','UNIPROT']
        substrate_table_id = [x for x in self.table_headers_dict if self.table_headers_dict[x][:5]==titles]
        return substrate_table_id
    
    def get_substrate_table_info(self):
        substrate_table_id = self._get_substrates_table_id()[0]
        hidden_divs = self.soup.find_all('div', id=re.compile('^tab{}'.format(substrate_table_id)))
        res_list = []
        for div in hidden_divs:
            # Extract values from the first, second, and fifth columns
            res_dict = {}
            for i,c in enumerate(self.table_headers_dict[substrate_table_id]):
                cell = div.find('div', class_='cell', id=lambda x: x and x.endswith('c{}'.format(i)))
                if cell:
                    value = cell.text.strip()
                    res_dict[c] = value
            if res_dict.keys():
                res_list.append(res_dict)
        self.substrate_table_info = res_list
    
#     def get_results_from_substrates_table(self):
#         get_substrates_table_id

In [ ]:
url = 'https://www.brenda-enzymes.org/enzyme.php?ecno=4.2.3.21#SUBSTRATE'
bp = BrendaPage(url)

In [ ]:
pd.DataFrame(bp.substrate_table_info)

In [ ]:
ec_numbers = list(brenda_flatfile['data'].keys())

In [ ]:
urls = ['https://www.brenda-enzymes.org/enzyme.php?ecno={}#SUBSTRATE'.format(ec) for ec in ec_numbers]

In [ ]:
brenda_pages_results = {}
for ec, url in tqdm(zip(ec_numbers[:10], urls[:10])):
    try:
        brenda_pages_results[ec] = BrendaPage(url).substrate_table_info
    except Exception as e:
        print (e, 'could not retrieve info for ec number {}'.format(ec))

In [ ]:
brenda_pages_results

In [ ]:
with open('test_scraped_brenda_substrate_results.json', 'w') as f:
    json.dump(brenda_pages_results,f)

In [ ]:
with open('test_scraped_brenda_substrate_results.json', 'r') as f:
    print(json.load(f))